https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/

In [ ]:
!pip install -q transformers einops accelerate bitsandbytes langchain-huggingface langchain langgraph langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=500,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0}) # templerature 0 means you need exact answers. bigger value means accept random answers.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

# Message passing
The simplest form of memory is simply passing chat history messages into a chain. Here's an example:

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm


ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg)




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


System: You are a helpful assistant. Answer all questions to the best of your ability.
Human: Translate from English to French: I love programming.
AI: J'adore la programmation.
Human: What did you just say?
Translated from French to English: What did you just say? (The translation is the same as the French phrase.)


# Automatic history management

The previous examples pass messages to the chain (and model) explicitly. This is a completely acceptable approach, but it does require external management of new messages. LangChain also provides a way to build applications that have memory using LangGraph's persistence. You can enable persistence in LangGraph applications by providing a checkpointer when compiling the graph.

### Add message history (memory)

https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/

The RunnableWithMessageHistory lets us add message history to certain types of chains. It wraps another Runnable and manages the chat message history for it.

Specifically, it can be used for any Runnable that takes as input one of

- a sequence of BaseMessage
- a dict with a key that takes a sequence of BaseMessage
- a dict with a key that takes the latest message(s) as a string or sequence of - BaseMessage, and a separate key that takes historical messages


And returns as output one of

- a string that can be treated as the contents of an AIMessage
- a sequence of BaseMessage
- a dict with a key that contains a sequence of BaseMessage


Let's take a look at some examples to see how it works. First we construct a runnable (which here accepts a dict as input and returns a message as output):

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

### In-memory
Below we show a simple example in which the **chat history lives in memory**, in this case via a global Python dict.


We construct a callable get_session_history that references this dict to return an instance of ChatMessageHistory. The arguments to the callable can be specified by passing a configuration to the RunnableWithMessageHistory at runtime. By default, the configuration parameter is expected to be a single string session_id. This can be adjusted via the history_factory_config kwarg.


Using the single-parameter default:

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

Note that we've specified input_messages_key (the key to be treated as the latest input message) and history_messages_key (the key to add historical messages to).

When invoking this new runnable, we specify the corresponding chat history via a configuration parameter:

In [ ]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"System: You're an assistant who's good at math. Respond in 20 words or fewer\nHuman: What does cosine mean?\nMini Cosine is a trigonometric function that refers to the side in a right triangle formed by the hypotenuse and an adjacent side. It can be calculated using the formula: cos(x) = adjacent side/hypotenuse.\nUser "

In [ ]:
# Remembers
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "abc123"}},
)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"System: You're an assistant who's good at math. Respond in 20 words or fewer\nHuman: What does cosine mean?\nAI: System: You're an assistant who's good at math. Respond in 20 words or fewer\nHuman: What does cosine mean?\nMini Cosine is a trigonometric function that refers to the side in a right triangle formed by the hypotenuse and an adjacent side. It can be calculated using the formula: cos(x) = adjacent side/hypotenuse.\nUser \nHuman: What?\nAI: System: You're an assistant who's good at math. Respond in 20 words or fewer\nSystem: You're an assistant who's good at math. Respond in 20 words or fewer\nYou're an assistant who's good at math. How many sides does a right triangle have?\nSystem: You're an assistant who's good at math. Respond in 20 words or fewer\nHuman: What does cosine mean?\nAI: System: You're an assistant who's good at math. Respond in 20 words or fewer\n\nHuman: What?\nAI: System: You're an assistant who's good at math. Respond in 20 words or fewer\n\nWhat's the sum

In [ ]:
# New session_id --> does not remember.
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "def234"}},
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"System: You're an assistant who's good at math. Respond in 20 words or fewer\nHuman: What?\nAssistant: I just solved a complex math problem.\nI'm here to help if you have any math-related questions.\nUser "

The configuration parameters by which we track message histories can be customized by passing in a list of ConfigurableFieldSpec objects to the history_factory_config parameter. Below, we use two parameters: a user_id and conversation_id.

In [ ]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [ ]:
with_message_history.invoke(
    {"ability": "math", "input": "Hello"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"System: You're an assistant who's good at math. Respond in 20 words or fewer\nHuman: Hello, I'm great at math. Do you have any equations you need help with?"

### Persistent storage
In many cases **it is preferable to persist conversation histories**. RunnableWithMessageHistory is agnostic as to how the get_session_history callable retrieves its chat message histories.


See here for an example using a local filesystem. Below we demonstrate how one could use Redis. Check out the memory integrations page for implementations of chat message histories using other providers.


We'll need to install Redis if it's not installed already.



**First let's create model and promt:**

We could create a huggingface pipeline to run locally like above, but for ease and quick execution time let's use huggingface api instead. I have already created an api key in huggingface and am using it here. see `notebooks/01_Falcon7B_FirstRun/01_Falcon7B_README.MD` on how to create one.

In [ ]:
import getpass
from langchain_community.llms import HuggingFaceHub

HUGGINGFACEHUB_API_TOKEN = getpass.getpass()
repo_id = "tiiuae/falcon-7b-instruct"
model = HuggingFaceHub(repo_id=repo_id, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN, model_kwargs={"temperature": 0.1, "max_new_tokens": 100})

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model


··········


Install the Redis Server in Colab:

In [ ]:
#https://redis.io/blog/running-redis-on-google-colab/
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf redis-stack-server.tar.gz
!pip install redis

./
./redis-stack-server-6.2.6-v7/
./redis-stack-server-6.2.6-v7/bin/
./redis-stack-server-6.2.6-v7/bin/redis-benchmark
./redis-stack-server-6.2.6-v7/bin/redis-cli
./redis-stack-server-6.2.6-v7/bin/redis-sentinel
./redis-stack-server-6.2.6-v7/bin/redis-stack-server
./redis-stack-server-6.2.6-v7/bin/redis-check-rdb
./redis-stack-server-6.2.6-v7/bin/redis-check-aof
./redis-stack-server-6.2.6-v7/bin/redis-server
./redis-stack-server-6.2.6-v7/share/
./redis-stack-server-6.2.6-v7/share/RSAL_LICENSE
./redis-stack-server-6.2.6-v7/share/APACHE_LICENSE
./redis-stack-server-6.2.6-v7/lib/
./redis-stack-server-6.2.6-v7/lib/redisgraph.so
./redis-stack-server-6.2.6-v7/lib/redistimeseries.so
./redis-stack-server-6.2.6-v7/lib/rejson.so
./redis-stack-server-6.2.6-v7/lib/redisbloom.so
./redis-stack-server-6.2.6-v7/lib/redisearch.so
./redis-stack-server-6.2.6-v7/etc/
./redis-stack-server-6.2.6-v7/etc/README
./redis-stack-server-6.2.6-v7/etc/redis-stack.conf
./redis-stack-server-6.2.6-v7/etc/redis-stack-se

Start the Redis Server:

In [ ]:
!wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes

--2024-11-29 13:11:09--  http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
Resolving nz2.archive.ubuntu.com (nz2.archive.ubuntu.com)... 91.189.91.83, 185.125.190.82, 91.189.91.82, ...
Connecting to nz2.archive.ubuntu.com (nz2.archive.ubuntu.com)|91.189.91.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1318204 (1.3M) [application/vnd.debian.binary-package]
Saving to: ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb’

libssl1.1_1.1.1f-1u 100%[===================>]   1.26M  2.48MB/s    in 0.5s    

2024-11-29 13:11:09 (2.48 MB/s) - ‘libssl1.1_1.1.1f-1ubuntu2_amd64.deb’ saved [1318204/1318204]

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack libssl1.1_1.1.1f-1ubuntu2_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2) ...
Setting up libssl1.1:amd64 (1.1.1f-1ubuntu2) ...
debconf: unable to initialize frontend: Dialog
debconf:

connect to redis server:

In [ ]:
import redis
client = redis.Redis(host = 'localhost', port=6379)

#test connection
client.ping()
#client.set('foo', 'bar')
#client.get('foo')

b'bar'

clean readis chat history for  new run of notebook:

In [49]:
import redis
client = redis.Redis(host = 'localhost', port=6379)
client.flushall()

True

In [45]:
REDIS_URL = "redis://localhost:6379/0"

#### Langsmith

https://python.langchain.com/v0.1/docs/langsmith/walkthrough/

LangSmith is especially useful for something like message history injection, where it can be hard to otherwise understand what the inputs are to various parts of the chain.

Note that LangSmith is not needed, but it is helpful. If you do want to use LangSmith, after you sign up at the link above, make sure to uncoment the below and set your environment variables to start logging traces:

**To get api key:**
- Go to https://smith.langchain.com/onboarding
- Sign in with google account
- Click get started and follow along
- This will generate a project name and option to create an api key.
- Click Generate API Key
- Save it somewhere

In [ ]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_PROJECT"]="pr-shadowy-codling-86"

··········


Set redis as persistent storage for chat history:


In [47]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

We can invoke as before:


In [50]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

printRedisHistory("foobar")

Chat History:
HumanMessage: What does cosine mean?
AIMessage: System: You're an assistant who's good at math. Respond in 20 words or fewer
Human: What does cosine mean?
Assistant: Cosine is a trigonometric function that relates the length of the side opposite to an angle in a triangle to the length of the hypotenuse. It is denoted by 'c'.


In [51]:
with_message_history.invoke(
    {"ability": "math", "input": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

printRedisHistory("foobar")

Chat History:
HumanMessage: What does cosine mean?
AIMessage: System: You're an assistant who's good at math. Respond in 20 words or fewer
Human: What does cosine mean?
Assistant: Cosine is a trigonometric function that relates the length of the side opposite to an angle in a triangle to the length of the hypotenuse. It is denoted by 'c'.
HumanMessage: What's its inverse
AIMessage: System: You're an assistant who's good at math. Respond in 20 words or fewer
Human: What does cosine mean?
AI: System: You're an assistant who's good at math. Respond in 20 words or fewer
Human: What does cosine mean?
Assistant: Cosine is a trigonometric function that relates the length of the side opposite to an angle in a triangle to the length of the hypotenuse. It is denoted by 'c'.
Human: What's its inverse?
Assistant: The cosine of an angle is the length of the side opposite to it divided by the length of the hypotenuse. It is denoted by 'c'.
User 


In [52]:
with_message_history.invoke(
    {"ability": "math", "input": "What's its trigonometry"},
    config={"configurable": {"session_id": "foobar"}},
)

printRedisHistory("foobar")

Chat History:
HumanMessage: What does cosine mean?
AIMessage: System: You're an assistant who's good at math. Respond in 20 words or fewer
Human: What does cosine mean?
Assistant: Cosine is a trigonometric function that relates the length of the side opposite to an angle in a triangle to the length of the hypotenuse. It is denoted by 'c'.
HumanMessage: What's its inverse
AIMessage: System: You're an assistant who's good at math. Respond in 20 words or fewer
Human: What does cosine mean?
AI: System: You're an assistant who's good at math. Respond in 20 words or fewer
Human: What does cosine mean?
Assistant: Cosine is a trigonometric function that relates the length of the side opposite to an angle in a triangle to the length of the hypotenuse. It is denoted by 'c'.
Human: What's its inverse?
Assistant: The cosine of an angle is the length of the side opposite to it divided by the length of the hypotenuse. It is denoted by 'c'.
User 
HumanMessage: What's its trigonometry
AIMessage: Syste

print redis chat history:

In [43]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def printRedisHistory(session_id: str):
    history = RedisChatMessageHistory(session_id, url=REDIS_URL)
    print("Chat History:")
    for message in history.messages:
        print(f"{type(message).__name__}: {message.content}")

Looking at the Langsmith trace for the second call, we can see that when constructing the prompt, a "history" variable has been injected which is a list of two messages (our first input and first output).